# Notes on extracting data from Tüba-D/Z

## Issue 1: coreference edges vs. coreferential edges

## Issue 2: extract all NPs (i.e. cat=NX or cat=N*?)

In [1]:
import os

import discoursegraphs as dg
import discourseinfostat as di

In [2]:
TUEBADZ8_FILE = dg.corpora.TUEBADZ_PATH

In [3]:
corpus = dg.read_exportxml(TUEBADZ8_FILE, debug=False)

In [4]:
doc = corpus.next()

In [5]:
debug_corpus = dg.read_exportxml(TUEBADZ8_FILE, debug=True)
debug_doc = debug_corpus.next()
edg = dg.readwrite.exportxml.ExportXMLDocumentGraph(debug_doc)

In [6]:
# %matplotlib inline
# %load_ext gvmagic

In [7]:
# %dotstr dg.print_dot(doc)

In [8]:
# for i, doc in enumerate(corpus):
#     try:
#         dg.print_dot(doc)
# #         print i, "sucess"
#     except:
#         print i, "failed"

In [9]:
# dg.info(doc)

In [10]:
import itertools

onset = 0
offset = 0

for token_id, token_str in itertools.islice(doc.get_tokens(), 5):
    offset = onset + len(token_str)
    print onset, token_str, offset
    doc.node[token_id]['{}:{}'.format(doc.ns, 'onset')] = onset
    doc.node[token_id]['{}:{}'.format(doc.ns, 'offset')] = offset
    print doc.node[token_id]
    onset = offset + 1

0 Veruntreute 11
{'layers': set(['exportxml', 'exportxml:token']), 'exportxml:offset': 11, 'form': 'Veruntreute', 'parent': 's1_500', 'morph': '3sit', 'pos': 'VVFIN', 'label': 'Veruntreute', 'lemma': 'veruntreuen', 'func': 'HD', 'exportxml:onset': 0, 'deprel': 'ROOT', 'exportxml:token': 'Veruntreute'}
12 die 15
{'layers': set(['exportxml', 'exportxml:token']), 'exportxml:offset': 15, 'form': 'die', 'parent': 's1_502', 'morph': 'nsf', 'pos': 'ART', 'label': 'die', 'lemma': 'die', 'dephead': 's1_3', 'func': '-', 'exportxml:onset': 12, 'deprel': 'DET', 'exportxml:token': 'die'}
16 AWO 19
{'layers': set(['exportxml', 'exportxml:token']), 'exportxml:offset': 19, 'parent': 's1_502', 'form': 'AWO', 'morph': 'nsf', 'pos': 'NN', 'label': 'AWO', 'lemma': 'AWO', 'dephead': 's1_1', 'func': 'HD', 'exportxml:onset': 16, 'deprel': 'SUBJ', 'exportxml:token': 'AWO'}
20 Spendengeld 31
{'layers': set(['exportxml', 'exportxml:token']), 'exportxml:offset': 31, 'form': 'Spendengeld', 'parent': 's1_503', 'mo

In [11]:
print dg.get_text(doc)

Veruntreute die AWO Spendengeld ? Staatsanwaltschaft muß AWO-Konten prüfen / Flossen 165.000 Mark Sammelgelder für Flutopfer in ein Altenheim in Danzig ? Landesvorsitzende Ute Wedemeier : Ein Buchungsfehler Im Januar hat die Arbeiterwohlfahrt Bremen ihren langjährigen Geschäftsführer Hans Taake fristlos entlassen , nun wird auch der Vorstand der Wohlfahrtsorganisation in den Fall hineingezogen . In einer anonymen Anzeige werden der Bremer Staatsanwaltschaft Details über dubiose finanzielle Transaktionen mitgeteilt . Verantwortlich , so das Schreiben einer Mitarbeiterin der AWO , sei die Landesvorsitzende Uter Wedemeier , die sich jetzt als " Sauberfrau " gebe , " wo doch alle wissen , wie eng sie mit Taake zusammenhing " . Vorwurf Nummer 1 : 165.000 Mark aus der bundesweiten Geldsammlung für die Flutopfer in Südpolen seien über das Konto des Bremer Landesverbandes der AWO an die Caritas in Danzig geflossen , " damit dort ein Altenheim gebaut wird " . Das Altenheim sei " ein Prestigeobj

# TODO: difference between `exportxml:coreference` and `exportxml:coreferential`?

The corpus metadata only has `coreferential`.  
I introduced `coreference` myself; this layer will be added  
to all `<relation>` with a `target` attribute, as well as all
`<splitRelation>`s.

```xml
<edge name="relation" parent="word|node">
  <enum-attr name="type">
   <val name="anaphoric" description="Anaphorisches Pronomen"/>
   <val name="cataphoric" description="Kataphorisches Pronomen"/>
   <val name="coreferential" description="Diskurs-altes nicht-Pronomen"/>
  </enum-attr>
  <node-ref name="target"/>
</edge>
```

In [12]:
len(dg.get_pointing_chains(doc, layer='exportxml:coreference'))

17

In [13]:
len(dg.get_pointing_chains(doc, layer='exportxml:coreferential'))

20

In [14]:
coreference_set = set(tuple(coref_list)
                      for coref_list in dg.get_pointing_chains(doc, layer='exportxml:coreference'))

In [15]:
coreferential_set = set(tuple(coref_list)
                      for coref_list in dg.get_pointing_chains(doc, layer='exportxml:coreferential'))

In [16]:
not_in_coreference_set = coreferential_set.difference(coreference_set)

In [17]:
not_in_coreferential_set = coreference_set.difference(coreferential_set)

In [18]:
for coref_chain in not_in_coreference_set:
    print ' -> '.join(dg.get_text(doc, token) for token in coref_chain)
    print

Ute Wedemeier -> Frau Wedemeier -> Die Landesvorsitzende Ute Wedemeier -> Wedemeier -> ich

der Prüfgesellschaft -> die

Landesverband -> Landesverbandes -> der Landesverband als Dachverband ohne hauptamtliches Personal -> des AWO-Landesverbandes -> des Landesverbandes der AWO -> des Landesverbandes -> des Landesverbandes -> des Bremer Landesverbandes der AWO -> ihren

Freuden-thal -> Karin Freudenthal

die Arbeiterwohlfahrt Bremen -> die AWO

Ute Wedemeier -> ihr

die AWO -> der Arbeiterwohlfahrt -> die AWO -> der AWO -> der AWO -> sich

Detlev Griesche und Karin Freudenthal -> den gewählten Revisoren des AWO-Landesverbandes

Koring -> Koring selbst -> seinen

Die AWO -> der AWO -> der AWO -> der Wohlfahrtsorganisation -> ihren

die Landesvorsitzende Uter Wedemeier -> Landesvorsitzende Ute Wedemeier

Tegeler -> seinen Bremerhavener Geschäftsführer Volker Tegeler -> Tegeler -> Tegeler -> Volker Tegeler , stellvertretender Geschäftsführer des Landesverbandes

Ute Wedemeier -> sie



In [19]:
for coref_chain in not_in_coreferential_set:
    print ' -> '.join(dg.get_text(doc, token) for token in coref_chain)
    print

ihn -> Tegeler -> seinen Bremerhavener Geschäftsführer Volker Tegeler -> Tegeler -> Tegeler -> Volker Tegeler , stellvertretender Geschäftsführer des Landesverbandes

die AWO -> der Arbeiterwohlfahrt -> die AWO -> der AWO -> der AWO -> sich -> Die AWO -> der AWO -> der AWO -> der Wohlfahrtsorganisation -> ihren -> die Arbeiterwohlfahrt Bremen -> die AWO

Landesverband -> Landesverbandes -> der Landesverband als Dachverband ohne hauptamtliches Personal -> des AWO-Landesverbandes -> des Landesverbandes der AWO -> des Landesverbandes -> des Landesverbandes -> des Bremer Landesverbandes der AWO -> ihren -> die Arbeiterwohlfahrt Bremen -> die AWO

Ute Wedemeier -> sie -> sie -> ihrer -> sie -> Ute Wedemeier -> Frau Wedemeier -> Die Landesvorsitzende Ute Wedemeier -> Wedemeier -> ich -> Die ehrenamtliche Landesvorsitzende Wedemeier -> sie -> sie -> Ute Wedemeier -> ihr -> sie -> die -> die Landesvorsitzende Uter Wedemeier -> Landesvorsitzende Ute Wedemeier

er -> Seine -> er -> er -> Koring 

# FAIL: both sets overlap and the chains in `coreference` often seem to subsume the ones from `coreferential`

This is how relations are specified in Tueba:

```xml
<edge name="relation" parent="word|node">
  <enum-attr name="type">
   <val name="anaphoric" description="Anaphorisches Pronomen"/>
   <val name="cataphoric" description="Kataphorisches Pronomen"/>
   <val name="coreferential" description="Diskurs-altes nicht-Pronomen"/>
  </enum-attr>
  <node-ref name="target"/>
</edge>
```

## TODO: find out where in the code `coreferential` edges are constructed


The parent node of a ``<relation>`` element will have a ``{'relation' : 'coreferential'}`` attribute,  
as well as an ``exportxml:coreferential`` layer.

```python
self.node[parent_node_id].update({'relation': reltype})
self.add_layer(parent_node_id, self.ns+':'+reltype)
```

for each ``<relation>`` with a ``target`` attribute,  
``add_relation()`` adds an edge with both its reltype  
as well as ``exportxml:coreference`` as layers.

```python
# reltype: anaphoric, cataphoric, coreferential
self.add_edge(parent_node_id, target_id,
              layers={self.ns, self.ns+':'+reltype,
                      self.ns+':coreference'},
              label=reltype,
              edge_type=dg.EdgeTypes.pointing_relation)
```

# Conclusion:

## stick with exportxml:coreference
## use exportxml:coreferential/anaphoric/cataphoric only for subclassification / error analysis

# TODO: generic way to extract NPs from corpora with dg?

- NPs have the node label `NX` in Tüba/DZ